In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Encoder + MLP
The idea of using an encoder is the denoise the data. After many attempts at using a unsupervised autoencoder, the choice landed on a bottleneck encoder as this will preserve the intra-feature relations. 

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
import random

import kerastuner as kt

def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    

## PurgedGroupTimeSeriesSplit
Click the code button to see. 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1,callbacks=None):
        val_losses = []
        for train_indices, test_indices in splits:
            X_train, X_test = [x[train_indices] for x in X], [x[test_indices] for x in X]
            y_train, y_test = [a[train_indices] for a in y], [a[test_indices] for a in y]
            if len(X_train) < 2:
                X_train = X_train[0]
                X_test = X_test[0]
            if len(y_train) < 2:
                y_train = y_train[0]
                y_test = y_test[0]
            
            model = self.hypermodel.build(trial.hyperparameters)
            hist = model.fit(X_train,y_train,
                      validation_data=(X_test,y_test),
                      epochs=epochs,
                        batch_size=batch_size,
                      callbacks=callbacks)
            
            val_losses.append([hist.history[k][-1] for k in hist.history])
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(trial.trial_id, {k:np.mean(val_losses[:,i]) for i,k in enumerate(hist.history.keys())})
        self.save_model(trial.trial_id, model)

### Loading the training data

In [ ]:
# TRAINING = False
# USE_FINETUNE = False     
# FOLDS = 5
# SEED = 1111

# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
# #train = pd.read_feather('../input/janestreet-save-as-feather/train.feather')

# train = train.query('date > 85').reset_index(drop = True) 
# train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
# train.fillna(train.mean(),inplace=True)
# train = train.query('weight > 0').reset_index(drop = True)
# #train['action'] = (train['resp'] > 0).astype('int')
# train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
# features = [c for c in train.columns if 'feature' in c]
# f_mean = np.mean(train[features[1:]].values,axis=0)

# resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

# #X = train[features].values
# #y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T #Multitarget

# f_mean = np.mean(train[features[1:]].values,axis=0)

# y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T


# features1 = [f'feature_{i}' for i in range(71)]
# features2 = [f'feature_{i}' for i in range(71, 130)]

# # X = [train.loc[:, features1].values, 
# #           train.loc[:, features2].values]

# assert len(features) == len(features1) + len(features2)

# #X = train.loc[:, train.columns.str.contains('feature')]



### Creating the autoencoder. 
The autoencoder should aid in denoising the data. Based on [this](https://www.semanticscholar.org/paper/Deep-Bottleneck-Classifiers-in-Supervised-Dimension-Parviainen/fb86483f7573f6430fe4597432b0cd3e34b16e43) paper. 

In [ ]:
def create_autoencoder(input_dim,output_dim,noise=0.05):
    i = Input(input_dim)
    encoded = BatchNormalization()(i)
    encoded = GaussianNoise(noise)(encoded)
    encoded = Dense(64,activation='relu')(encoded)
    decoded = Dropout(0.2)(encoded)
    decoded = Dense(input_dim,name='decoded')(decoded)
    x = Dense(32,activation='relu')(decoded)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(output_dim,activation='sigmoid',name='label_output')(x)
    
    encoder = Model(inputs=i,outputs=decoded)
    autoencoder = Model(inputs=i,outputs=[decoded,x])
    
    autoencoder.compile(optimizer=Adam(0.001),loss={'decoded':'mse','label_output':'binary_crossentropy'})
    return autoencoder, encoder

### Creating the MLP. 

In [ ]:
def create_model(hp,input_dim,output_dim,encoder):
    inputs = Input(input_dim)
    
    x = encoder(inputs)
    x = Concatenate()([x,inputs]) #use both raw and encoded features
    x = BatchNormalization()(x)
    x = Dropout(hp.Float('init_dropout',0.0,0.5))(x)
    
    for i in range(hp.Int('num_layers',1,3)):
        x = Dense(hp.Int('num_units_{i}',64,256))(x)
        x = BatchNormalization()(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(hp.Float(f'dropout_{i}',0.0,0.5))(x)
    x = Dense(output_dim,activation='sigmoid')(x)
    model = Model(inputs=inputs,outputs=x)
    model.compile(optimizer=Adam(hp.Float('lr',0.00001,0.1,default=0.001)),loss=BinaryCrossentropy(label_smoothing=hp.Float('label_smoothing',0.0,0.1)),metrics=[tf.keras.metrics.AUC(name = 'auc')])
    return model

In [ ]:
from tensorflow.keras import backend as K
class Mish(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs):
        return inputs * K.tanh(K.softplus(inputs))

    def get_config(self):
        base_config = super(Mish, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

def mish(x):
	return tf.keras.layers.Lambda(lambda x: x*K.tanh(K.softplus(x)))(x)

tf.keras.utils.get_custom_objects().update({'mish': tf.keras.layers.Activation(mish)})

In [ ]:
def create_resnet(hp,n_features, n_labels, label_smoothing = 0.0005):    
    input_1 = tf.keras.layers.Input(shape = (n_features,), name = 'Input1')
    #input_2 = tf.keras.layers.Input(shape = (n_features,), name = 'Input2')
    input_11 = tf.keras.backend.square(input_1)
    

    head_1 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(hp.Float('init_dropout',0.1,0.5)),
        tf.keras.layers.Dense(hp.Int('num_units_{i}',64,512), activation="mish"), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(hp.Float('init_dropout',0.1,0.5)),
        tf.keras.layers.Dense(128, activation = "mish")
        ],name='Head1') 

    input_3 = head_1(input_1)
    input_31 = head_1(input_11)
    
    
    
    
    
    x = tf.keras.layers.Dense(4096, activation='relu')(input_1)
    x = tf.keras.layers.Reshape((128, 32))(x)
    x = tf.keras.layers.Conv1D(filters=32,
                      kernel_size=7,
                      strides=1,
                      activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    input_311 = tf.keras.layers.Flatten()(x)
    
    
    
    
    
    input_3_concat = tf.keras.layers.Concatenate()([input_1, input_3, input_11, input_31, input_311])

    head_2 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(hp.Float('init_dropout',0.1,0.5)),
        tf.keras.layers.Dense(hp.Int('num_units_{i}',64,512), "mish"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(hp.Float('init_dropout',0.1,0.5)),
        tf.keras.layers.Dense(128, "mish"),
        ],name='Head2')

    input_4 = head_2(input_3_concat)
    input_4_avg = tf.keras.layers.Concatenate()([input_1, input_3, input_4, input_11, input_31, input_311])
    #tf.keras.layers.Average()([input_3, input_4]) 

    head_3 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(hp.Int('num_units_{i}',64,512), kernel_initializer='lecun_normal', activation='mish'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(hp.Float('init_dropout',0.1,0.5)),
        tf.keras.layers.Dense(hp.Int('num_units_{i}',64,512), kernel_initializer='lecun_normal', activation='mish'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(hp.Float('init_dropout',0.1,0.5)),
        tf.keras.layers.Dense(n_labels, activation="sigmoid")
        ],name='Head3')

    output = head_3(input_4_avg)


    model = tf.keras.models.Model(inputs = input_1, outputs = output)
    model.compile(optimizer=Adam(hp.Float('lr',0.00001,0.01,default=0.001)), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing), metrics=['AUC'])
    
    return model

#clf = create_resnet(len(features1), len(features2), len(resp_cols), 1e-02)

#clf.summary()

### Defining and training the autoencoder. 
We add gaussian noise with mean and std from training data. After training we lock the layers in the encoder from further training. 

In [ ]:
# autoencoder, encoder = create_autoencoder(X.shape[-1],y.shape[-1],noise=0.1)
# set_all_seeds(SEED)    
# if TRAINING:
#     autoencoder.fit(X,(X,y),
#                     epochs=2,
#                     batch_size=4096, 
#                     validation_split=0.1,
#                     callbacks=[EarlyStopping('val_loss',patience=1,restore_best_weights=True)])
#     encoder.save_weights('./encoder.hdf5')
# else:
#     encoder.load_weights('../input/v7seedencoderdecoder2223/encoder.hdf5')
# encoder.trainable = False

### Running CV
Following [this notebook](https://www.kaggle.com/gogo827jz/jane-street-ffill-xgboost-purgedtimeseriescv) which use 5 PurgedGroupTimeSeriesSplit split on the dates in the training data. 

We add the locked encoder as the first layer of the MLP. This seems to help in speeding up the submission rather than first predicting using the encoder then using the MLP. 

We use a Baysian Optimizer to find the optimal HPs for out model. 20 trials take about 2 hours on GPU. 

In [ ]:
train = pd.read_feather('../input/janestreet-save-as-feather/train.feather')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train.fillna(train.mean(),inplace=True)



In [ ]:

train['action'] = ((train['resp'].values) > 0).astype(int)


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

#y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T



In [ ]:
y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

In [ ]:
from tensorflow.keras.layers import MaxPooling1D, Dense, LeakyReLU, Conv1D 
leaky_relu_alpha =0.05
LeakyReLU(alpha=leaky_relu_alpha)

In [ ]:

x_train = X.values.reshape(-1, X.shape[1], 1)  # reshaping for convnet

print(x_train.shape[1:])

X = x_train


In [ ]:
from tensorflow import keras
def createcnn(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()
    model.add(Conv1D(hp.Int('num_units_{i}',64,256), 2, input_shape=x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=leaky_relu_alpha))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(hp.Float('init_dropout',0.1,0.5)))  
    model.add(Flatten()) 
    
    model.add(Dense(hp.Int('num_units2_{j}',64,256)))
    model.add(LeakyReLU(alpha=leaky_relu_alpha))
    model.add(Dropout(hp.Float('init_dropout2',0.1,0.5) ))  
    model.add(Dense(5))
    model.add(Activation("sigmoid"))

    model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),
                  loss=BinaryCrossentropy(label_smoothing=0.095),
                  metrics=[tf.keras.metrics.AUC(name = 'auc'), "accuracy"])

    return model

In [ ]:
def createcnn():  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()
    model.add(Conv1D(100, 2, input_shape=x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=leaky_relu_alpha))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.4))  
    model.add(Flatten()) 
    
    model.add(Dense(96))
    model.add(LeakyReLU(alpha=leaky_relu_alpha))
    model.add(Dropout(0.1 ))  
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=leaky_relu_alpha))
    model.add(Dropout(0.2 )) 
    model.add(Dense(5))
    model.add(Activation("sigmoid"))

    model.compile(optimizer=keras.optimizers.Adam(lr=1e-4),
                  loss=BinaryCrossentropy(label_smoothing=0.025),
                  metrics=[tf.keras.metrics.AUC(name = 'auc'), "accuracy"])

    return model

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import MaxPooling1D, Dense, LeakyReLU, Conv1D 
from tensorflow.keras.layers import Flatten, Activation, BatchNormalization, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
import pickle

In [ ]:
from tensorflow import keras
# model_fn = lambda hp:  createcnn(hp)

# tuner = CVTuner(
#         hypermodel=model_fn,
#         oracle=kt.oracles.BayesianOptimization(
#         objective= kt.Objective('val_auc', direction='max'),
#         num_initial_points=4,
#         max_trials=10))

FOLDS = 2
#FOLDS = 1
SEED = 1111
TRAINING = False
USE_FINETUNE = True  
set_all_seeds(SEED)   



if TRAINING:
    models = []
    gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
    splits = list(gkf.split(y, groups=train['date'].values))
    
    tuner.search((X,),(y,),splits=splits,batch_size=4096,epochs=1000,callbacks=[EarlyStopping('val_auc', mode='max',patience=5)])
    hp  = tuner.get_best_hyperparameters(1)[0]
    pd.to_pickle(hp,f'./best_hp_{SEED}.pkl')
    
    model = model_fn(hp)
    
    model.fit(X,y ,epochs=40,batch_size=4096)
    model.save_weights(f'./model_{SEED}_.hdf5')

    models.append(model)
    

    tuner.results_summary()
# else:
#     models = []
#     hp = pd.read_pickle(f'../input/v7seedencoderdecoder2223/best_hp_{SEED}.pkl')
#     for f in range(FOLDS):
#         model = model_fn(hp)
#         if USE_FINETUNE:
#             model.load_weights(f'../input/v7seedencoderdecoder2223/model_{SEED}_{f}_finetune.hdf5')
#         else:
#             model.load_weights(f'../input/v7seedencoderdecoder2223/model_{SEED}_{f}.hdf5')
#         models.append(model)
        
        
        
else:

    
    models = []
        
    
    #hp = pd.read_pickle(f'../input/newcnnonlinetrainnewmoretrain/best_hp_1111.pkl')
    #model = createcnn(hp)
    model = createcnn()
    model.summary()
    
    #model.load_weights(f'/kaggle/input/d/aimind/newcnnonlinetrainnewmoretrain/model_1111_.tf')
    
    model.fit(X,y,validation_split=0.01 ,epochs=64,batch_size=4096)    #########   80  ###########################################
    model.save_weights(f'./model_{SEED}_.tf')
    #model = LiteModel.from_keras_model(model)
  
    models.append(model)
    

## Submission

In [ ]:
if not TRAINING:
    f = np.mean
    #models = models[-2:]
    import janestreet
    env = janestreet.make_env()
    th = 0.5
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt[:, 1:].sum()):
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
            pred = f(pred)
            pred_df.action = np.where(pred >= th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)